In [36]:
import pandas as pd
import numpy as np
from datetime import datetime

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt

from scipy import stats
from statistics import mode

In [56]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [82]:
import json
import pickle
from flask import  Flask, request, jsonify, render_template
from sklearn.metrics import r2_score

In [90]:
# Iniciamos nuestra API
app = Flask('Emision de CO2')

In [91]:
# Entrenamos el modelo

@app.route('/entrenar_modelo',methods=['POST'])
def entrenar_modelo():
    
    # la función "request.get_json" de Flask para capturar la información que le envíemos a la API
    data = request.get_json(force=True)
    
    # Separamos la información de data y usamos json.loads() para transformar el dataframe que está en formato
    # json a un diccionario y luego lo convertimos en un DataFrame.
    df=pd.DataFrame(json.loads(data['base']))
    columns_name=data['lista_predictores']
    target_name=data['target']
    directorio_modelo=data['directory']
    nombre_modelo=data['name']
        
    # Separamos las "X" y la "y" para entrenar nuestro modelo
    X = df[columns_name]
    y = df[target_name]
    
    # Hacemos el train test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=False, random_state=40)
    
    # Instaciamos el modelo de regresion Lineal.
    modelo = LinearRegression()
    
    # Entrenamos el modelo 
    modelo.fit(X_train, y_train)
        
    # Evaluamos el R2 de train y de test
    result_train=r2_score(y_train,modelo.predict(X_train))
    result_test=r2_score(y_test,modelo.predict(X_test))
        
    # Guardamos el modelo entrenado en una carpeta que se llama "modelos" que esté a la misma altura que la notebook
    dir_=directorio_modelo+'/'+nombre_modelo+'.pkl'
    with open(dir_, 'wb') as modelo_pkl:
        pickle.dump(modelo, modelo_pkl)
       
    
    # La función devuelva el alpha elegido por el modelo, el resultado de train y el de test.
    return jsonify({'r2_resultado_train':float(result_train), 'r2_resultado_test':float(result_test)})

In [ ]:
# Iniciamos nuestro segunda función asociada a su endpoint. Usamos el método POST ya que vamos a enviar información
# al servidor: la figura con los coeficientes de nuestro modelo entrenado
@app.route('/plot_coeficientes',methods=['POST'])
def plotear_coeficientes():
    
    # la función "request.get_json" de Flask para capturar la información que le envíemos a la API
    data = request.get_json(force=True)
    
    # Separamos la información de data.
    direccion=data['direccion']
    name_modelo=data['nombre_modelo']
    colores=data['paleta_colores']
    size=data['tamano']
    predictores=data['predictores']
    
    # Levantamos el modelo que tenemos grabado en disco con el llamado anterior
    # (hay que tener en cuenta que tenemos que hacer el llamado anterior primero cuando estemos en la notebook
    # que simula ser el cliente).
    dir_input=direccion+'/'+name_modelo+'.pkl'
    with open(dir_input, 'rb') as modelo_pkl:
        modelo_load = pickle.load(modelo_pkl)
        
    # Levantamos los coeficientes ya calculados del modelo
    # Los ponemos en un dataframe junto con el nombre de los predictores.
    # Pasamos los coeficientes a valores absolutos y los ordenamos de mayor a 
    # menor para hacer el gráfico.
    coef=abs(pd.DataFrame({'coeficientes':modelo_load.coef_},index=predictores)).\
                           sort_values(by='coeficientes',ascending=False)
    
    
    # Generamos la figura y la guardamos
    fig, ((ax1)) = plt.subplots(1,1,gridspec_kw={'hspace': 0.45, 'wspace': 0.15},figsize=size)
    fig.suptitle("Coeficientes",y=0.96,x=0.135,fontsize=24,fontweight='bold')

    ax1 = sns.barplot(x="index", y="coeficientes", data=coef.reset_index(),ax=ax1,palette=colores)
    ax1.xaxis.set_label_text('Coeficientes')
    ax1.yaxis.set_label_text('Valores coeficientes')
    plt.close(); #usamos esta línea para que la figura no se imprima en pantalla
    
    # guardamos la figura donde se guardan los modelos
    dir_figura=direccion+'/'+name_modelo+'.jpg'
    fig.savefig(dir_figura,dpi=150)
    
    # devolvemos un json con los coeficientes. Dado que las API devuelven la información en formato
    # json, usamos el método de los dataframes ".to_json()" para poder retornar los datos de los
    # coeficientes.
    return jsonify(coef.to_json())    

In [92]:
# Ahora ejecutamos esta línea de código que pone a disposición los tres endpoints que armamos arriba. 
# Es hora de ir a la otra notebook en la que simulamos ser un cliente y hacer los llamadados para cada
# uno de estos tres endopoints...
app.run(host='0.0.0.0')

 * Serving Flask app "Emision de CO2" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [14/Mar/2021 09:42:37] "POST /entrenar_modelo HTTP/1.1" 200 -
127.0.0.1 - - [14/Mar/2021 09:42:42] "POST /entrenar_modelo HTTP/1.1" 200 -


In [ ]:
# Dado que en este endopint no vamos a guardar nada en el servidor, sino recibir información, usamos 
# el método GET. 
@app.route("/prediccion",methods=['GET'])
def predecir_puntaje():
    
    # Usamos request.args para tomar las query que le pasamos a la URL
    direccion=request.args['direccion']
    name_modelo=request.args['nombre_modelo']
    name_scaler=request.args['nombre_scalador']
    caso_to_predict=request.args['features']
    
    # Levantamos el modelo y el escalador que ya tenemos entrenado
    with open(direccion+'/'+name_modelo+'.pkl', 'rb') as modelo_pkl:
        modelo_load = pickle.load(modelo_pkl)
        
    with open(direccion+'/'+name_scaler+'.pkl', 'rb') as scaler_pkl:
        scaler_load = pickle.load(scaler_pkl)
        
    # re-escalamos los datos con el escalador entrenado (tengan en cuenta que el escalador va a estar
    # esperando una estructura como un DataFrame)
    scaled_case=scaler_load.transform(pd.DataFrame(json.loads(caso_to_predict),index=[0]))
    
    # realizamos la prediccion
    prediccion=modelo_load.predict(scaled_case)
    
    return jsonify({'prediccion':float(prediccion)})